<a href="https://colab.research.google.com/github/Ronimol23/AaxAudioConverter/blob/master/RFM%20utilising%20rfm_norm%20to%20create%20a%20stnadardised%20scale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.cluster import KMeans
#loading file path
%cd /content/drive/MyDrive/Segmentation-20220613T115039Z-001/Segmentation/RFM


/content/drive/MyDrive/Segmentation-20220613T115039Z-001/Segmentation/RFM


In [2]:
# load transaction data into a DataFrame
df = pd.read_csv('online_shoppers.csv', 
                 encoding= 'ISO-8859-1').dropna()
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [3]:
# change name InvoiceDate to date
df.rename(columns={'InvoiceDate': 'date',
                   'CustomerID': 'customer_id'
}, inplace=True)
df.head(0)

,InvoiceNo,StockCode,Description,Quantity,date,UnitPrice,customer_id,Country


In [4]:
#Creating amount variable, Quantity x UnitPrice
df['amount'] = df['Quantity'] * df['UnitPrice']
df.head(0)

,InvoiceNo,StockCode,Description,Quantity,date,UnitPrice,customer_id,Country,amount


In [5]:
# calculate RFM values for each customer
# rfm = df.groupby('customer_id').agg({
#     'date': lambda x: (pd.Timestamp.now() - x.max()).days, # recency
#     'customer_id': 'count', # frequency
#     'amount': 'sum' # monetary
# })

rfm = df.groupby('customer_id').agg({
    'date': lambda x: (pd.Timestamp.now() - pd.to_datetime(x.max())).days, # recency
    'customer_id': 'count', # frequency
    'amount': 'sum' # monetary
})


In [6]:
rfm.head()

,date,customer_id,amount
customer_id,,,
12346.0,4416,2,0.00
12347.0,4220,182,4310.00
12348.0,4165,31,1797.24
12349.0,4109,73,1757.55
12350.0,4400,17,334.40


In [7]:
# rename columns for clarity
rfm.rename(columns={'date': 'recency', 'customer_id': 'frequency', 
                    'amount': 'monetary'}, inplace=True)
rfm.head()

,recency,frequency,monetary
customer_id,,,
12346.0,4416,2,0.00
12347.0,4220,182,4310.00
12348.0,4165,31,1797.24
12349.0,4109,73,1757.55
12350.0,4400,17,334.40


In [12]:
# normalize RFM values to scale between 0 and 1
rfm_norm = (rfm - rfm.min()) / (rfm.max() - rfm.min())
rfm_norm.head()

,recency,frequency,monetary,cluster
customer_id,,,,
12346.0,0.871314,0.000125,0.015109,0.0
12347.0,0.345845,0.022676,0.030297,1.0
12348.0,0.198391,0.003758,0.021442,0.5
12349.0,0.048257,0.009020,0.021303,0.5
12350.0,0.828418,0.002005,0.016288,0.0


In [14]:
# fit K-Means model to the normalized RFM values
kmeans = KMeans(n_clusters=3, random_state=0).fit(rfm_norm)
print(kmeans)

KMeans(n_clusters=3, random_state=0)


In [10]:
# add cluster labels to the DataFrame
rfm['cluster'] = kmeans.labels_

In [11]:
# print the resulting RFM clusters
print(rfm.groupby('cluster').mean())

             recency   frequency     monetary
cluster                                      
0        4388.535055   35.116851   529.545166
1        4159.776126  123.274002  2687.969330
2        4256.353527   73.105394  1279.722648
